#### The purpose of this notebook is to produce train and test data for the ML algorithm using template pulses

In [1]:
import ROOT as r
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as skl
import pickle
import spliland
import importlib
import random
import fclParse
from scipy import stats

Welcome to JupyROOT 6.18/04


In [2]:
# run this to make printing cleaner
np.set_printoptions(precision=None)

In [1]:
# Reload stuff in case you make changes!

importlib.reload(spliland);

NameError: name 'importlib' is not defined

In [2]:
f = r.TFile("./calotemplate15.root")
noiseDict = fclParse.fclReader("gm2pedestals_run3formatRE.fcl")
enCalDict = fclParse.fclReader("mipEnergyCalibration_PostDisk_CoincidencNumber_2.fcl")

NameError: name 'r' is not defined

In [10]:
nIslands = 2000

x = []
y = []
x_delta = []

for index in range(0, nIslands+1):
    xtalNum = str(np.random.randint(low=0, high=54))
    thisNoise = noiseDict['pedestalConstantsLaserRun3']['calo15']['xtal' + xtalNum]['noiseLevel']
    thisEnCal = enCalDict['absolute_calibration_constants']['calo15']['xtal' + xtalNum]

    indexSpline = spliland.Spline(f.Get("masterSpline_xtal" + xtalNum), xtalNum, 15)
    indexIsland = spliland.Island(referenceSpline=indexSpline,
                                  minPulses = 1, maxPulses = 4,
                                  useUniformEscale = False,
                                  energyCalibrationVal = thisEnCal,
                                  energyPeak = 0, energyScale = 600,
                                  minEscale = 1, maxEscale=1,
                                  deltaTmin=0, deltaTmax=25,
                                  minTimeOffset=3,
                                  verbosity=False,
                                  normalize=True,
                                  chop = False,
                                  chopThreshold = 50,
                                  nPreSamples = 8, nPostSamples = 18,
                                  noise=True, noiseLevel=thisNoise,
                                  randomizeTailLength=False)

    x.append([indexIsland.time, indexIsland.yValues])
    y.append(indexIsland.nPulses)
    x_delta.append([indexIsland.time, indexIsland.simple_ys])

    if (index % 1000 == 0):
        print("index {0}".format(index))

with open('./template_data/set_7/x_test.data', 'wb') as xFile:
    pickle.dump(x, xFile)
    xFile.close()

with open('./template_data/set_7/x_delta_test.data', 'wb') as x_deltaFile:
    pickle.dump(x_delta, x_deltaFile)
    x_deltaFile.close()

with open('./template_data/set_7/y_test.data', 'wb') as yFile:
    pickle.dump(y, yFile)
    yFile.close()

index 0
index 1000
index 2000
